In [2]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
import cv2
import math
import json
import itertools as it

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

from kaggle_camera_model_id_lib.utils import PechkaBot, ImageList, NpzFolder, NCrops, TifFolder, MultiDataset
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead, IEEEfcn, ResNetFC, ResNetX, FatNet1
from kaggle_camera_model_id_lib.models import InceptionResNetV2fc, InceptionResNetV2fcSmall
from kaggle_camera_model_id_lib.utils import jpg_compress, equalize_v_hist, hsv_convert
from kaggle_camera_model_id_lib.utils import scale_crop_pad, gamma_correction
from kaggle_camera_model_id_lib.utils import patch_quality_dich, n_random_crops, n_pseudorandom_crops
from kaggle_camera_model_id_lib.models import DANet, ResNetFeatureExtractor, AvgFcClassifier, FCDiscriminator
from kaggle_camera_model_id_lib.models import AvgClassifier, InceptionResNetV2, ResNetDense, ResNetDenseFC

import imgaug as ia
from imgaug import augmenters as iaa

import pretrainedmodels as pm

In [2]:
crop_center_size = 1520
crop_size = 256
random_crop = transforms.RandomCrop(crop_size)
center_crop = transforms.CenterCrop(crop_center_size)
rvf = transforms.RandomVerticalFlip()
rhf = transforms.RandomHorizontalFlip()
random_flip = lambda img: rvf(rhf(img))

n_crops_search_train = 36
n_crops_train = 9

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

In [3]:
train_list_path = '/home/mephistopheies/storage2/data/camera-model-id/train.tsv'
val_path = '/home/mephistopheies/storage2/data/camera-model-id/val/'
test_path = '/home/mephistopheies/storage2/data/camera-model-id/raw/test/'

In [3]:
X_var = Variable(torch.from_numpy(np.random.uniform(-1, 1, size=(5, 3, 64, 64)).astype(np.float32)))

In [2]:
x = [1.2, 1.4, 0.5]

In [2]:
n_classes = 10

model = ResNetDenseFC(
            models.resnet.BasicBlock, [3, 4, 6, 3], num_classes=n_classes, load_resnet='resnet34', 
            zero_first_center=True)

In [4]:
x = model.conv1(X_var)
x = model.bn1(x)
x = model.relu(x)
x = model.maxpool(x)

x1 = model.layer1(x)
x2 = model.layer2(x1)
x3 = model.layer3(x2)
x4 = model.layer4(x3)

print(x1.shape)
print(x2.shape)
print(x3.shape)
print(x4.shape)

torch.Size([5, 64, 16, 16])
torch.Size([5, 128, 8, 8])
torch.Size([5, 256, 4, 4])
torch.Size([5, 512, 2, 2])


In [5]:
x1 = model.maxpool8(x1)
x2 = model.maxpool4(x2)
x3 = model.maxpool2(x3)

print(x1.shape)
print(x2.shape)
print(x3.shape)
print(x4.shape)

torch.Size([5, 64, 2, 2])
torch.Size([5, 128, 2, 2])
torch.Size([5, 256, 2, 2])
torch.Size([5, 512, 2, 2])


In [6]:
len(NCrops(np.random.uniform(0, 1, size=(512, 512, 3)), 64, 32))

225